In [ ]:
# Import các thư viện cần thiết
import numpy as np
import cv2
import pandas as pd
import os
import pickle
from google.colab import drive
from google.colab.patches import cv2_imshow
from sklearn.model_selection import train_test_split

In [ ]:
# Connect Google Drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/test_opencv'
# Hiển thị tất cả thư mục hoặc file
os.listdir(path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['model',
 'face1.jpg',
 'face2.jpg',
 '30.ipynb',
 'Untitled0.ipynb',
 'FaceImage',
 '27.ipynb',
 'data_face_features.pickle',
 '31_data_preprocessing.ipynb',
 '28.ipynb',
 '29.ipynb',
 'Untitled1.ipynb']

In [ ]:
# 1. DATA


In [ ]:
# Load data from pickle file
data = pickle.load(open("/content/drive/MyDrive/test_opencv/data_face_features.pickle", mode="rb"))

In [ ]:
x = np.array(data['data'])
y = np.array(data['label'])

print(x)
print(y)

[[[ 0.12031338  0.07270519 -0.07699942 ... -0.0756839   0.03358966
   -0.02564145]]

 [[ 0.10534605  0.09276143 -0.05423581 ... -0.05424381  0.07689524
   -0.01417274]]

 [[ 0.07846182  0.13807029 -0.04713358 ... -0.02009326  0.09488688
   -0.03312805]]

 ...

 [[ 0.06350037  0.19478916  0.00305566 ...  0.06762125  0.11532869
    0.03716978]]

 [[ 0.06529244  0.14482117 -0.04807781 ...  0.05233255  0.00911174
    0.02666707]]

 [[ 0.065717    0.15750356 -0.04845994 ... -0.03104515  0.07409232
    0.08183732]]]
['1721031591_NguyenHongNhung' '1721031591_NguyenHongNhung'
 '1721031591_NguyenHongNhung' ... '151901205_BuiKhanhBinh'
 '151901205_BuiKhanhBinh' '151901205_BuiKhanhBinh']


In [ ]:
x.shape


(4694, 1, 128)

In [ ]:
y.shape


(4694,)

In [ ]:
x = x.reshape(-1, 128)


In [ ]:
x.shape


(4694, 128)

In [ ]:
# chia nhỏ dữ liệu
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, random_state=0)

In [ ]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((3755, 128), (939, 128), (3755,), (939,))

In [ ]:
# 2. Train machine learning


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score

In [ ]:
# Logistic regression
model_logistic = LogisticRegression()
model_logistic.fit(x_train, y_train) # training

LogisticRegression()

In [ ]:

def get_report(model, x_train, y_train, x_test, y_test):
  y_pred_train = model.predict(x_train)
  y_pred_test = model.predict(x_test)

  # accuracy score
  acc_train = accuracy_score(y_train, y_pred_train)
  acc_test = accuracy_score(y_test, y_pred_test)

  # f1 score
  f1_score_train = f1_score(y_train, y_pred_train, average="macro")
  f1_score_test = f1_score(y_test, y_pred_test, average="macro")

  # print
  print('Accurency Train = %0.2f'%acc_train)
  print('Accurency Test = %0.2f'%acc_test)
  print('F1 Score Train = %0.2f'%f1_score_train)
  print('F1 Score Test = %0.2f'%f1_score_test)

In [ ]:
get_report(model_logistic, x_train, y_train, x_test, y_test)


Accurency Train = 0.71
Accurency Test = 0.70
F1 Score Train = 0.51
F1 Score Test = 0.49


In [ ]:
# Support Vector Machines
model_svc = SVC(probability=True)
model_svc.fit(x_train, y_train)

SVC(probability=True)

In [ ]:
get_report(model_svc, x_train, y_train, x_test, y_test)


Accurency Train = 0.91
Accurency Test = 0.84
F1 Score Train = 0.82
F1 Score Test = 0.75


In [ ]:
# Random forest
model_rf = RandomForestClassifier(n_estimators=20,)
model_rf.fit(x_train, y_train)

RandomForestClassifier(n_estimators=20)

In [ ]:
get_report(model_rf, x_train, y_train, x_test, y_test)


Accurency Train = 1.00
Accurency Test = 0.76
F1 Score Train = 1.00
F1 Score Test = 0.67


In [ ]:
# Voting Classifier
model_voting = VotingClassifier(estimators=[
    ('logistic', LogisticRegression()),
    ('svm', SVC(probability=True)),
    ('rf', RandomForestClassifier(n_estimators=20,)),
], voting='soft', weights=[2,3,1])

In [ ]:
model_voting.fit(x_train, y_train)


VotingClassifier(estimators=[('logistic', LogisticRegression()),
                             ('svm', SVC(probability=True)),
                             ('rf', RandomForestClassifier(n_estimators=20))],
                 voting='soft', weights=[2, 3, 1])

In [ ]:
get_report(model_voting, x_train, y_train, x_test, y_test)


Accurency Train = 0.98
Accurency Test = 0.83
F1 Score Train = 0.97
F1 Score Test = 0.74


In [ ]:
# 3. Parameter Tuning
from sklearn.model_selection import GridSearchCV

model_grid = GridSearchCV(model_voting, param_grid={
    'svm__C': [3,5,7,10],
    'svm__gamma': [0.1, 0.3, 0.5],
    'rf__n_estimators': [5, 10,20],
    'rf__max_depth': [3, 5, 7],
    'voting': ['soft', 'hard']
}, scoring='accuracy', cv=3, n_jobs=1, verbose=2)


In [ ]:
model_grid.fit(x_train, y_train)


Fitting 3 folds for each of 216 candidates, totalling 648 fits
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=  14.8s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=  15.3s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   7.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   7.8s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   6.1s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   8.4s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   5.6s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   7.4s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   5.7s
[

GridSearchCV(cv=3,
             estimator=VotingClassifier(estimators=[('logistic',
                                                     LogisticRegression()),
                                                    ('svm',
                                                     SVC(probability=True)),
                                                    ('rf',
                                                     RandomForestClassifier(n_estimators=20))],
                                        voting='soft', weights=[2, 3, 1]),
             n_jobs=1,
             param_grid={'rf__max_depth': [3, 5, 7],
                         'rf__n_estimators': [5, 10, 20],
                         'svm__C': [3, 5, 7, 10], 'svm__gamma': [0.1, 0.3, 0.5],
                         'voting': ['soft', 'hard']},
             scoring='accuracy', verbose=2)

In [ ]:
model_best_estimator = model_grid.best_estimator_


In [ ]:
model_grid.best_score_


0.8407504028725902

In [ ]:
# 4. Save Model
pickle.dump(model_best_estimator, open("/content/drive/MyDrive/test_opencv/ml_face_person_identity.pkl", mode='wb'))
